In [1]:
import sys
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import deepxde as dde
from deepxde.backend import tf

2024-08-05 17:33:51.142448: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 17:33:51.549596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 17:33:51.549631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 17:33:51.639674: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 17:33:51.738278: I tensorflow/core/platform/cpu_feature_guar

Instructions for updating:
non-resource variables are not supported in the long term


Enable just-in-time compilation with XLA.



In [2]:

def get_data(filename, n_train, n_test):
    data = scipy.io.loadmat(filename)
    x = data["x_coor"].astype(np.float32)
    y = data["y_coor"].astype(np.float32)
    t = data["t"].astype(np.float32)
    Vx = data["Vx"].astype(np.float32)
    Vy = data["Vy"].astype(np.float32)
    
    u = np.stack((Vx, Vy), axis=-1)  # Shape: (N, k, n_points, 2)
    u_final = u[:, -1, :, :]  # Shape: (N, n_points, 2)

    u_flattened = u.reshape(u.shape[0], u.shape[1], -1) #Input branch only recieves the 1D array
    # Branch: initial velocity field
    branch = u_flattened[:, 0, :]  # Shape: (N, n_points, 2)
    # Trunk: spatial coordinates grid
    xy = np.vstack((np.ravel(x), np.ravel(y))).T  # N x 2 (x, y)：grid points
    
    # Output: velocity field at the final time step


    # Split into training and testing data
    branch_train = branch[:n_train]
    xy_train = xy
    u_final_train = u_final[:n_train]

    branch_test = branch[-n_test:]
    xy_test = xy
    u_final_test = u_final[-n_test:]

    return (branch_train, xy_train), u_final_train, (branch_test, xy_train), u_final_test

def main():
    n_train = 100  # Adjust based on your dataset size
    n_test = 60  # Adjust based on your dataset size
    nx = 6400

    data_path = "/home/liruixiang/data/gepup/train_data_new.mat"
    x_train, u_final_train, x_test, u_final_test = get_data(data_path, n_train, n_test)
    
    # Split u_final_train and u_final_test into Vx and Vy parts
    u_final_train_vx = u_final_train[:, :, 0]
    u_final_train_vy = u_final_train[:, :, 1]
    u_final_test_vx = u_final_test[:, :, 0]
    u_final_test_vy = u_final_test[:, :, 1]
    
    # train_x = [branch_train, xy_train]
    # test_x = [branch_test, xy_test]

    # Create datasets for Vx and Vy
    data_vx = dde.data.Triple(
        x_train, u_final_train_vx,
        x_test, u_final_test_vx
    )
    data_vy = dde.data.Triple(
        x_train, u_final_train_vy,
        x_test, u_final_test_vy
    )

    # 创建两个子网络，分别生成Vx和Vy
    net_vx = dde.maps.DeepONetCartesianProd(
        [nx * 2, 128, 128], [2, 128, 128], "relu", "Glorot normal"
    )

    net_vy = dde.maps.DeepONetCartesianProd(
        [nx * 2, 128, 128], [2, 128, 128], "relu", "Glorot normal"
    )

    # 创建模型
    model_vx = dde.Model(data_vx, net_vx)
    model_vy = dde.Model(data_vy, net_vy)

    # 编译模型
    model_vx.compile(
        "adam",
        lr=1e-3,
        decay=("inverse time", 1, 1e-4),
        metrics=["mean l2 relative error"],
    )
    model_vy.compile(
        "adam",
        lr=1e-3,
        decay=("inverse time", 1, 1e-4),
        metrics=["mean l2 relative error"],
    )

    # 训练模型
    losshistory_vx, train_state_vx = model_vx.train(epochs=250000, batch_size=None)
    losshistory_vy, train_state_vy = model_vy.train(epochs=250000, batch_size=None)

    # 预测
    y_pred_vx = model_vx.predict(data_vx.test_x)
    y_pred_vy = model_vy.predict(data_vy.test_x)

    # 保存预测结果
    np.savetxt("y_pred_vx_deeponet.dat", y_pred_vx[0].reshape(nx, 1))
    np.savetxt("y_pred_vy_deeponet.dat", y_pred_vy[0].reshape(nx, 1))
    np.savetxt("y_true_vx_deeponet.dat", data_vx.test_y[0].reshape(nx, 1))
    np.savetxt("y_true_vy_deeponet.dat", data_vy.test_y[0].reshape(nx, 1))
    np.savetxt("y_error_vx_deeponet.dat", (y_pred_vx[0] - data_vx.test_y[0]).reshape(nx, 1))
    np.savetxt("y_error_vy_deeponet.dat", (y_pred_vy[0] - data_vy.test_y[0]).reshape(nx, 1))

if __name__ == "__main__":
    # Enable memory growth for GPUs
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    if physical_devices:
        try:
            for device in physical_devices:
                tf.config.experimental.set_memory_growth(device, True)
        except RuntimeError as e:
            print(e)
    
    main()


Compiling model...
Building DeepONetCartesianProd...


/home/liruixiang/.conda/envs/deeponet/lib/python3.9/site-packages/deepxde/nn/tensorflow_compat_v1/deeponet.py:549: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  y_func = tf.layers.dense(
/home/liruixiang/.conda/envs/deeponet/lib/python3.9/site-packages/deepxde/nn/tensorflow_compat_v1/deeponet.py:556: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  y_func = tf.layers.dense(
/home/liruixiang/.conda/envs/deeponet/lib/python3.9/site-packages/deepxde/nn/tensorflow_compat_v1/deeponet.py:570: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  y_loc = tf.layers.dense(


'build' took 0.602472 s



2024-08-05 17:34:36.940074: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-08-05 17:34:36.940675: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16936 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6


'compile' took 3.700605 s

Compiling model...
Building DeepONetCartesianProd...
'build' took 0.502772 s



...

'compile' took 2.204762 s



2024-08-05 17:34:40.948371: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


Training model...



2024-08-05 17:34:43.247817: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc79c0092f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-05 17:34:43.247864: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-08-05 17:34:43.345162: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-05 17:34:44.138575: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1722850489.241190  133471 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Step      Train loss    Test loss     Test metric   
0         [8.21e-01]    [8.41e-01]    [1.02e+00]    
1000      [9.31e-02]    [9.98e-02]    [4.32e-01]    
2000      [8.37e-02]    [9.00e-02]    [4.13e-01]    
3000      [8.06e-02]    [8.69e-02]    [4.03e-01]    
4000      [7.87e-02]    [8.47e-02]    [3.92e-01]    
5000      [7.65e-02]    [8.14e-02]    [3.84e-01]    
6000      [6.18e-02]    [6.49e-02]    [3.50e-01]    
7000      [5.61e-02]    [5.86e-02]    [3.37e-01]    
8000      [4.83e-02]    [5.09e-02]    [3.16e-01]    
9000      [4.25e-02]    [4.55e-02]    [2.95e-01]    
10000     [3.97e-02]    [4.27e-02]    [2.83e-01]    
11000     [3.79e-02]    [4.08e-02]    [2.77e-01]    
12000     [3.60e-02]    [3.89e-02]    [2.69e-01]    
13000     [3.63e-02]    [3.91e-02]    [2.67e-01]    
14000     [3.25e-02]    [3.51e-02]    [2.55e-01]    
15000     [3.12e-02]    [3.37e-02]    [2.49e-01]    
16000     [3.14e-02]    [3.38e-02]    [2.48e-01]    
17000     [2.88e-02]    [3.11e-02]    [2.39e-0